In [1]:
import os
import pickle
import cv2
import numpy as np
from functools import cmp_to_key
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf

from tensorflow import keras
%matplotlib inline 
import matplotlib.pyplot as plt

In [2]:
def extractSymbols(imgOrig, showSteps = False):
    debugImgSteps = []
    imgGray = cv2.cvtColor(imgOrig,cv2.COLOR_BGR2GRAY)
    imgFiltered = cv2.medianBlur(imgGray, 5)
    debugImgSteps.append(imgFiltered)
    
    imgCanny = cv2.Canny(imgFiltered, 50,180)
    debugImgSteps.append(imgCanny)

    kernel = np.ones((5,5), np.uint8)
    imgDilated = cv2.dilate(imgCanny, kernel, iterations=5)
    debugImgSteps.append(imgDilated)

    contours, _= cv2.findContours(imgDilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    boundingBoxes = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        boundingBoxes.append((x,y,w,h))

    global rowsG
    rowsG, _, _ = imgOrig.shape
    key_leftRightTopBottom = cmp_to_key(leftRightTopBottom)
    boundingBoxes = sorted(boundingBoxes, key=key_leftRightTopBottom)

    symbols = []
    for (i, box) in enumerate(boundingBoxes):
        x,y,w,h = box
        mathSymbol = imgOrig[y:y+h, x:x+w]
        mathSymbol = cv2.cvtColor(mathSymbol, cv2.COLOR_BGR2GRAY) #converting to Gray as tensorflow deals with grayscale or RGB, not BGR
        mathSymbol = cv2.resize(mathSymbol, (45,45), interpolation=cv2.INTER_AREA) #to have the same size as trained images in the dataset
        debugImgSteps.append(mathSymbol)
        mathSymbolF = mathSymbol.astype('float32') #optional: tensorflows deals with float32, not uint8
        tf.keras.utils.normalize(mathSymbolF, axis=1)
        symbols.append(mathSymbolF)

    if showSteps:
        dispImages(debugImgSteps)

    return symbols
        

In [3]:
def leftRightTopBottom(tup1, tup2):
    x1, y1, _, _ = tup1
    x2, y2, _, _ = tup2
    rows = rowsG
    yRegion1, yRegion2 = -1, -1

    for i in range(4):
        if y1 < rows/4 + rows*(i/4):
            yRegion1 = i
            break
    else:
        if yRegion1 == -1:
            yRegion1 = 4

    for i in range(4):
        if y2 < rows/4 + rows*(i/4):
            yRegion2 = i
            break
    else:
        if yRegion2 == -1:
            yRegion2 = 4
    
    if yRegion1 < yRegion2:
        return -1
    elif yRegion2 < yRegion1:
        return 1
    elif x1 <= x2:
        return -1
    else:
        return 1


In [4]:
def dispImages(imgs):
    for img in imgs:
        cv2.imshow('Image', img)
        cv2.waitKey(0)
    else:
        cv2.destroyAllWindows()

In [5]:
img = cv2.imread('tests/test16.png')
symbols = extractSymbols(img, showSteps=True)

## Normalizing image pixels

In [6]:
dic = {
    "decimal":r".",
    "div":r"\div",
    "eight": r"8",
    "equal": r"=",
    "five": r"5",
    "four": r"4",
    "minus": r"-",
    "nine": r"9",
    "one": r"1",
    "plus cleaned": r"+",
    "seven": r"7",
    "six": r"6",
    "three": r"3",
    "times": r"\times",
    "two": r"2",
    "zero": r"0",

}

# Preparing the dataset

## Reading the kaggle [dataset](https://www.kaggle.com/datasets/xainano/handwrittenmathsymbols?resource=download)

Steps:
1. create a list of images and another list of labels for each image
2. store them in pickle files for easy retrieval when re-running the code 

In [7]:
def loadData(dataDir):
    imgs = []
    labels = []
    for key, value in dic.items():
        path = os.path.join(dataDir, key)
        print(path)
        for imgName in os.listdir(path):
            try:
                img = cv2.imread(os.path.join(path, imgName), cv2.COLOR_BGR2GRAY) 
                imgs.append(img)
                labels.append(value)
            except Exception as e:
                print(e)    
    return (imgs, labels)

The following cell is commented as it takes a long time (10min if image RGB, 1min otherwise) to create the pickle files

In [8]:
imgs, labels = loadData('mathSymbolsDataset2/train/')

mathSymbolsDataset2/train/decimal
mathSymbolsDataset2/train/div
mathSymbolsDataset2/train/eight
mathSymbolsDataset2/train/equal
mathSymbolsDataset2/train/five
mathSymbolsDataset2/train/four
mathSymbolsDataset2/train/minus
mathSymbolsDataset2/train/nine
mathSymbolsDataset2/train/one
mathSymbolsDataset2/train/plus cleaned
mathSymbolsDataset2/train/seven
mathSymbolsDataset2/train/six
mathSymbolsDataset2/train/three
mathSymbolsDataset2/train/times
mathSymbolsDataset2/train/two
mathSymbolsDataset2/train/zero


In [9]:
with open("x_symbols_2.pickle", 'wb') as f:
    pickle.dump(imgs, f)
with open("y_latex_2.pickle", 'wb') as f:
    pickle.dump(labels, f)

In [10]:
with open("x_symbols_2.pickle", 'rb') as f:
    imgs = pickle.load(f)
with open("y_latex_2.pickle", 'rb') as f:
    labels = pickle.load(f)

In [ ]:
def loadData(dataDir):
    imgs = []
    labels = []
    for key, value in dic.items():
        path = os.path.join(dataDir, key)
        print(path)
        for imgName in os.listdir(path):
            try:
                img = cv2.imread(os.path.join(path, imgName), cv2.COLOR_BGR2GRAY) 
                imgs.append(img)
                labels.append(value)
            except Exception as e:
                print(e)    
    return (imgs, labels)

In [ ]:
def loadData(dataDir):
    imgs = []
    labels = []
    for key, value in dic.items():
        path = os.path.join(dataDir, key)
        print(path)
        for imgName in os.listdir(path):
            try:
                img = cv2.imread(os.path.join(path, imgName), cv2.COLOR_BGR2GRAY) 
                imgs.append(img)
                labels.append(value)
            except Exception as e:
                print(e)    
    return (imgs, labels)

In [ ]:
def loadData(dataDir):
    imgs = []
    labels = []
    for key, value in dic.items():
        path = os.path.join(dataDir, key)
        print(path)
        for imgName in os.listdir(path):
            try:
                img = cv2.imread(os.path.join(path, imgName), cv2.COLOR_BGR2GRAY) 
                imgs.append(img)
                labels.append(value)
            except Exception as e:
                print(e)    
    return (imgs, labels)

In [11]:
latexToNums = {k: v for v, k in enumerate(np.unique(labels))}
#this dictionary is to revert the predicted numeric code back to latex: 
numsToLatex = {v: k for v, k in enumerate(np.unique(labels))}
latexToNums

{'+': 0,
 '-': 1,
 '.': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 '=': 13,
 '\\div': 14,
 '\\times': 15}

In [12]:
labelsNums = [latexToNums[label] for label in labels]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(imgs, labelsNums, test_size=0.33, stratify=labelsNums, random_state=42)

In [28]:
x_train = np.array(x_train)


In [33]:
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1) 

AxisError: axis 1 is out of bounds for array of dimension 1

In [22]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [25]:
x_train[1].size

60750

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2025, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2025, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.softmax))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [15]:
model = keras.models.load_model("NNModel")

In [17]:
print(np.argmax(model.predict(symbols[1].reshape(1,45,45))))

2
